In [1]:
import numpy as np
import pandas as pd 
import os 
import matplotlib.pyplot as plt
import xml.etree.ElementTree as xet
import cv2

In [2]:
df  = pd.read_csv('D:/M. Tech/Companies/Projects/Automatic Number Plate Recognition/labels.csv')
df.head()

,filepath,xmin,ymin,xmax,ymax
0,D:/M. Tech/Companies/Projects/Automatic Number...,226,125,419,173
1,D:/M. Tech/Companies/Projects/Automatic Number...,134,128,262,160
2,D:/M. Tech/Companies/Projects/Automatic Number...,140,5,303,148
3,D:/M. Tech/Companies/Projects/Automatic Number...,175,114,214,131
4,D:/M. Tech/Companies/Projects/Automatic Number...,167,202,240,220


- Based on the filepath data find the filename in that filepath of text file 

In [3]:
filename = df['filepath'][0]
filename

'D:/M. Tech/Companies/Projects/Automatic Number Plate Recognition/annotations\\Cars0.xml'

In [4]:
xet.parse(filename).getroot().find('filename').text

'Cars0.png'

In [5]:
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('D:/M. Tech/Companies/Projects/Automatic Number Plate Recognition/images',filename_image)
    return filepath_image
getFilename(filename)    

'D:/M. Tech/Companies/Projects/Automatic Number Plate Recognition/images\\Cars0.png'

In [6]:
image_path = df['filepath'].apply(getFilename)
image_path

0      D:/M. Tech/Companies/Projects/Automatic Number...
1      D:/M. Tech/Companies/Projects/Automatic Number...
2      D:/M. Tech/Companies/Projects/Automatic Number...
3      D:/M. Tech/Companies/Projects/Automatic Number...
4      D:/M. Tech/Companies/Projects/Automatic Number...
                             ...                        
428    D:/M. Tech/Companies/Projects/Automatic Number...
429    D:/M. Tech/Companies/Projects/Automatic Number...
430    D:/M. Tech/Companies/Projects/Automatic Number...
431    D:/M. Tech/Companies/Projects/Automatic Number...
432    D:/M. Tech/Companies/Projects/Automatic Number...
Name: filepath, Length: 433, dtype: object

## Verify image and output

In [7]:
file_path =  image_path[0]
file_path

'D:/M. Tech/Companies/Projects/Automatic Number Plate Recognition/images\\Cars0.png'

In [8]:
img = cv2.imread(file_path)
cv2.namedWindow('example', cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
#find the bounding box is correct or not
cv2.rectangle(img,(226,125),(419,173),(0,255,0),3)
cv2.namedWindow('example', cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Data Preprocessing

In [10]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [11]:
labels = df.iloc[:,1:].values

In [12]:
data = [] #to store normalised image arr 
output = [] # to store normalised max min value

for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h, w,d = img_arr.shape

    #preprocessing
    laod_image = load_img(image, target_size = (224,224))
    load_image_arr = img_to_array(laod_image)
    #Normalise image arr
    norm_load_image_arr = load_image_arr/255.0
    #normalise image labels 
    xmin, xmax, ymin, ymax = labels[ind]
    nxmin, nxmax = xmin / w, xmax / w
    nymin, nymax = ymin / h, ymax / h
    label_norm = (nxmin, nxmax, nymin, nymax)
    data.append(norm_load_image_arr)
    output.append(label_norm)

#normalization to labels

xmin, xmax, ymin, ymax = labels[ind]
nxmin, nxmax = xmin / w, xmax / w
nymin, nymax = ymin / h, ymax / h

In [13]:
norm_load_image_arr

array([[[0.9490196 , 0.96862745, 0.9843137 ],
        [0.65882355, 0.6862745 , 0.7176471 ],
        [0.5764706 , 0.6313726 , 0.6745098 ],
        ...,
        [0.39607844, 0.43529412, 0.47058824],
        [0.34901962, 0.3882353 , 0.42352942],
        [0.36862746, 0.40784314, 0.44313726]],

       [[0.9764706 , 0.98039216, 0.9882353 ],
        [0.9019608 , 0.9137255 , 0.93333334],
        [0.7372549 , 0.76862746, 0.80784315],
        ...,
        [0.3882353 , 0.44313726, 0.4862745 ],
        [0.3254902 , 0.38039216, 0.42352942],
        [0.34509805, 0.4       , 0.44313726]],

       [[0.9843137 , 0.98039216, 0.972549  ],
        [0.99607843, 0.99607843, 1.        ],
        [0.9607843 , 0.98039216, 0.99607843],
        ...,
        [0.3882353 , 0.45882353, 0.5137255 ],
        [0.32156864, 0.39215687, 0.44705883],
        [0.36078432, 0.43137255, 0.4862745 ]],

       ...,

       [[0.38039216, 0.31764707, 0.1882353 ],
        [0.3764706 , 0.3137255 , 0.18431373],
        [0.37254903, 0

In [14]:
label_norm

(0.18666666666666668, 0.528, 0.524, 0.542)

In [15]:
X = np.array(data, dtype=np.float32)
y = np.array(output, dtype=np.float32)
X.shape, y.shape

((433, 224, 224, 3), (433, 4))

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8, random_state=0)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((346, 224, 224, 3), (87, 224, 224, 3), (346, 4), (87, 4))

 # Deep Learning Model
 Transfer learning from tensor flow 2.x

In [17]:
from tensorflow.keras.applications import MobileNetV2, InceptionV3, InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.models import Model
import tensorflow as tf

In [28]:
inception_resnet = InceptionResNetV2(weights='imagenet', include_top=False
                                      , input_tensor=Input(shape=(224,224,3)))
inception_resnet.trainable=False

#------------------------------------
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500, activation='relu')(headmodel)
headmodel = Dense(250, activation='relu')(headmodel)
headmodel = Dense(4, activation='sigmoid')(headmodel)
#-----------model-----------------
model = Model(inputs=inception_resnet.input,outputs=headmodel)

In [29]:
model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_203 (Conv2D)         (None, 111, 111, 32)         864       ['input_2[0][0]']             
                                                                                                  
 batch_normalization_203 (B  (None, 111, 111, 32)         96        ['conv2d_203[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_203 (Activation  (None, 111, 111, 32)         0         ['batch_normalization_20

                                                                                                  
 conv2d_214 (Conv2D)         (None, 25, 25, 64)           12288     ['average_pooling2d_1[0][0]'] 
                                                                                                  
 batch_normalization_208 (B  (None, 25, 25, 96)           288       ['conv2d_208[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 batch_normalization_210 (B  (None, 25, 25, 64)           192       ['conv2d_210[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 batch_normalization_213 (B  (None, 25, 25, 96)           288       ['conv2d_213[0][0]']          
 atchNorma

                                                                                                  
 block35_1_conv (Conv2D)     (None, 25, 25, 320)          41280     ['block35_1_mixed[0][0]']     
                                                                                                  
 custom_scale_layer_40 (Cus  (None, 25, 25, 320)          0         ['mixed_5b[0][0]',            
 tomScaleLayer)                                                      'block35_1_conv[0][0]']      
                                                                                                  
 block35_1_ac (Activation)   (None, 25, 25, 320)          0         ['custom_scale_layer_40[0][0]'
                                                                    ]                             
                                                                                                  
 conv2d_224 (Conv2D)         (None, 25, 25, 32)           10240     ['block35_1_ac[0][0]']        
          

                                                                                                  
 batch_normalization_231 (B  (None, 25, 25, 48)           144       ['conv2d_231[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_228 (Activation  (None, 25, 25, 32)           0         ['batch_normalization_228[0][0
 )                                                                  ]']                           
                                                                                                  
 activation_231 (Activation  (None, 25, 25, 48)           0         ['batch_normalization_231[0][0
 )                                                                  ]']                           
                                                                                                  
 conv2d_22

 atchNormalization)                                                                               
                                                                                                  
 activation_233 (Activation  (None, 25, 25, 32)           0         ['batch_normalization_233[0][0
 )                                                                  ]']                           
                                                                                                  
 activation_235 (Activation  (None, 25, 25, 32)           0         ['batch_normalization_235[0][0
 )                                                                  ]']                           
                                                                                                  
 activation_238 (Activation  (None, 25, 25, 64)           0         ['batch_normalization_238[0][0
 )                                                                  ]']                           
          

...

                                                                                                  
 batch_normalization_373 (B  (None, 5, 5, 256)            768       ['conv2d_373[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_370 (Activation  (None, 5, 5, 192)            0         ['batch_normalization_370[0][0
 )                                                                  ]']                           
                                                                                                  
 activation_373 (Activation  (None, 5, 5, 256)            0         ['batch_normalization_373[0][0
 )                                                                  ]']                           
                                                                                                  
 block8_2_

                                                                                                  
 batch_normalization_378 (B  (None, 5, 5, 192)            576       ['conv2d_378[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 batch_normalization_381 (B  (None, 5, 5, 256)            768       ['conv2d_381[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_378 (Activation  (None, 5, 5, 192)            0         ['batch_normalization_378[0][0
 )                                                                  ]']                           
                                                                                                  
 activatio

 conv2d_386 (Conv2D)         (None, 5, 5, 192)            399360    ['block8_5_ac[0][0]']         
                                                                                                  
 conv2d_389 (Conv2D)         (None, 5, 5, 256)            172032    ['activation_388[0][0]']      
                                                                                                  
 batch_normalization_386 (B  (None, 5, 5, 192)            576       ['conv2d_386[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 batch_normalization_389 (B  (None, 5, 5, 256)            768       ['conv2d_389[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activatio

 activation_396 (Activation  (None, 5, 5, 224)            0         ['batch_normalization_396[0][0
 )                                                                  ]']                           
                                                                                                  
 conv2d_394 (Conv2D)         (None, 5, 5, 192)            399360    ['block8_7_ac[0][0]']         
                                                                                                  
 conv2d_397 (Conv2D)         (None, 5, 5, 256)            172032    ['activation_396[0][0]']      
                                                                                                  
 batch_normalization_394 (B  (None, 5, 5, 192)            576       ['conv2d_394[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 batch_nor

 batch_normalization_404 (B  (None, 5, 5, 224)            672       ['conv2d_404[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_404 (Activation  (None, 5, 5, 224)            0         ['batch_normalization_404[0][0
 )                                                                  ]']                           
                                                                                                  
 conv2d_402 (Conv2D)         (None, 5, 5, 192)            399360    ['block8_9_ac[0][0]']         
                                                                                                  
 conv2d_405 (Conv2D)         (None, 5, 5, 256)            172032    ['activation_404[0][0]']      
                                                                                                  
 batch_nor

# model training

In [30]:
from tensorflow.keras.callbacks import TensorBoard
tfb = TensorBoard('object_detection')

In [31]:
history = model.fit(x=X_train, y=y_train, batch_size =32, epochs= 20,
                   validation_data=(X_test,y_test),callbacks=[tfb])

Epoch 1/20
11/11 [==============================] - 127s 10s/step - loss: 0.1319 - val_loss: 0.1241
Epoch 2/20
11/11 [==============================] - 93s 9s/step - loss: 0.1123 - val_loss: 0.1127
Epoch 3/20
11/11 [==============================] - 92s 9s/step - loss: 0.1071 - val_loss: 0.1086
Epoch 4/20
11/11 [==============================] - 101s 9s/step - loss: 0.1028 - val_loss: 0.1065
Epoch 5/20
11/11 [==============================] - 95s 9s/step - loss: 0.1007 - val_loss: 0.1052
Epoch 6/20
11/11 [==============================] - 92s 8s/step - loss: 0.0999 - val_loss: 0.1054
Epoch 7/20
11/11 [==============================] - 105s 10s/step - loss: 0.1001 - val_loss: 0.1073
Epoch 8/20
11/11 [==============================] - 100s 9s/step - loss: 0.1011 - val_loss: 0.1077
Epoch 9/20
11/11 [==============================] - 96s 9s/step - loss: 0.1006 - val_loss: 0.1071
Epoch 10/20
11/11 [==============================] - 92s 9s/step - loss: 0.0998 - val_loss: 0.1074
Epoch 11/20
1

In [ ]:
# history = model.fit(x=X_train, y=y_train, batch_size =32, epochs= 200,
#                    validation_data=(X_test,y_test),callbacks=[tfb],initial_epoch = 21)

In [32]:
model.save('D:/M. Tech/Companies/Projects/Automatic Number Plate Recognition/models/object_detection.h5')

C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
